In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt

In [2]:
humana = pd.read_csv("HMAHCC_COMP.csv")

//anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [59]:
# Extracting History of AUD

humana_2_1 = humana[humana.event_descr == ('Fully Paid Claim' or 'Surgery' or 'New provider')]#events relevant to AUD/SUD diagnosis

humana_2_1 = humana_2_1.loc[:,['id','event_attr1']]    #selecting diagnosis only, which is attribute #1

humana_2_2 = humana[humana.event_descr == ('RX Claim - Rejected')]    #pulling out separately due to different attribute

humana_2_2 = humana_2_2.loc[:,['id','event_attr2']]    #selecting diagnosis, which is attribute #2 for rx claim rejected


In [63]:
# Create a 2-column dataframe for manipulation

humana_2_2.rename(columns={"id": "id", "event_attr2": "event_attr1"},inplace = True)
humana_2 = humana_2_1.append([humana_2_2])
humana_3 = pd.DataFrame.copy(humana_2)

humana_2.rename(columns={"id": "id", "event_attr1": "AUD_Incidence"},inplace = True)
humana_2 = humana_2.fillna('NO VALUE')

In [64]:
# Scan for alcohol-related records

humana_2 = humana_2[humana_2["AUD_Incidence"].str.contains("ALCOHOL")]
humana_22 = humana_2[humana_2["AUD_Incidence"].str.contains("NONALCOHOLIC")] # Exclude NASH etc.

idx1 = set(humana_2.set_index(['id', "AUD_Incidence"]).index)  # This two lines remove duplicate as well
idx2 = set(humana_22.set_index(['id', "AUD_Incidence"]).index)

humana_2 = pd.DataFrame(list(idx1 - idx2), columns=humana_2.columns) 

In [65]:
# Aggregate records for frequency in the 4-year time period

feature_2 = humana_2.groupby('id').count()
print(feature_2)

               AUD_Incidence
id                          
ID1002482139               1
ID1008136105               2
ID10201747254              3
ID10629089274              1
ID10748737263              1
...                      ...
ID99564407113              1
ID99770796764              2
ID998126948                1
ID99819880491              2
ID99866353021              1

[428 rows x 1 columns]


In [69]:
# Extracting Disability Status, similar to above

# Create a 2-column dataframe for manipulation

humana_3.rename(columns={"id": "id", "event_attr1": "Disability Status"},inplace = True)
humana_3 = humana_3.fillna('NO VALUE')

# Scan for disability-related records

humana_3 = humana_3[humana_3["Disability Status"].str.contains("DISABILITY")]
humana_33 = humana_3[humana_3["Disability Status"].str.contains("DISABLE")]

idx1 = set(humana_3.set_index(['id', "Disability Status"]).index)  # This two lines remove duplicate as well
idx2 = set(humana_33.set_index(['id', "Disability Status"]).index)

humana_3 = pd.DataFrame(list(idx1 - idx2), columns=humana_3.columns) 

# Aggregate records for frequency in the 4-year time period

feature_3 = humana_3.groupby('id').count()
print(feature_3)

               Disability Status
id                              
ID20722312303                  1
ID24983128404                  1
ID29980463176                  1
ID40650138370                  1
ID6262058860                   1
ID79797676242                  1
ID83015627732                  1
ID94186818364                  1


In [102]:
# Extracting Beneficiaries’ residence

humana_5 = humana[humana.event_descr == ('Inbound Call by Mbr')]#events relevant to Caller's residence

humana_5 = humana_5.loc[:,['id','event_attr5']]    #selecting location only, which is attribute #5
humana_5['event_attr5'].value_counts()

# As shown below, dataset includes patients from KY, AZ, AK, OH, TX, WI, NC, UT, WA, PR, ID, AL, FL

X, KY                  105147
LOUISVILLE, KY          43918
PHOENIX, AZ             21554
0000, AK                 6196
CINCINNATI, OH           5844
-, AZ                    3899
WACO, TX                 3741
TUCSON, AZ               2700
-, WI                    2087
KILEEN, NC               2084
SALT LAKE CITY, UT       1666
-, KY                     658
LOUSIVILLE, KY            596
0000, WA                  505
00000, AK                 421
TEMPE, AZ                 350
LOU, KY                   337
XXX, KY                   329
PUERTO RICO, PR           301
9999999999999, WI         298
LOUISVILLE, AK            217
DE PERE, WI               203
TUSCON, AZ                171
TWIN FALLS, ID            156
999999999999999, WI       145
CINCI, OH                 125
CARY, NC                  124
GREEN BAY, WI              93
99999999999999, WI         90
9999999999999, TX          66
9, KY                      64
9, AZ                      59
NEW YORK, AK               51
SAN JUAN, 

In [108]:
# Scan for different states in US

pf = humana_5['event_attr5'].str.split(',', expand=True)
real_humana_5 = pd.DataFrame.copy(humana_5)
real_humana_5['event_attr5'] = pf[1]
real_humana_5 = real_humana_5.dropna()

feature_5 = real_humana_5.groupby('id')['event_attr5'].agg(pd.Series.mode).to_frame()
feature_5.rename(columns={"id": "id", "event_attr5": "Residence"},inplace = True)

print(feature_5)
feature_5['Residence'].value_counts()

              Residence
id                     
ID10010854159        KY
ID10013863216        KY
ID10024447278        KY
ID1002482139         KY
ID10036289882        KY
...                 ...
ID99971049558        KY
ID99975648894        KY
ID99985278397        KY
ID99987501651        KY
ID99994197161        KY

[11836 rows x 1 columns]


 KY           10284
 AZ             215
 OH             206
 AK             161
 TX              62
              ...  
[ AZ,  KY]        1
[ AZ,  KY]        1
[ KY,  PR]        1
[ AZ,  KY]        1
[ AZ,  KY]        1
Name: Residence, Length: 748, dtype: int64